In [1]:
import pandas as pd
import json

json_file = 'population_data.json'
with open(json_file, 'r') as file:
    json_data = json.load(file)
population_df = pd.json_normalize(json_data['data'])
print(population_df.head())

  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2022  2022   331097593  united-states
1   01000US  United States     2021  2021   329725481  united-states
2   01000US  United States     2020  2020   326569308  united-states
3   01000US  United States     2019  2019   324697795  united-states
4   01000US  United States     2018  2018   322903030  united-states


In [2]:
filtered_df = population_df[(population_df['ID Year'] >= 2013) & (population_df['ID Year'] <= 2018)]
mean = filtered_df['Population'].mean()
std = filtered_df['Population'].std()
print(f'Population mean: {mean}, Standard deviation: {std}')

Population mean: 317437383.0, Standard deviation: 4257089.5415293295


In [3]:
bls_df = pd.read_csv('pr.data.0.Current', sep='\t')
bls_df.columns = bls_df.columns.str.strip()
bls_df = bls_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
print(bls_df.head())

     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6            NaN
1  PRS30006011  1995    Q02    2.1            NaN
2  PRS30006011  1995    Q03    0.9            NaN
3  PRS30006011  1995    Q04    0.1            NaN
4  PRS30006011  1995    Q05    1.4            NaN


In [4]:
# Report the best year by series ID.

summed_df = bls_df.groupby(['series_id','year'])['value'].sum().reset_index()
best_values = summed_df.groupby(['series_id'])['value'].transform('max')
best_value_df = summed_df[summed_df['value'] == best_values]
best_value_df.style.format(precision=3)

,series_id,year,value
27,PRS30006011,2022,20.500
57,PRS30006012,2022,17.100
63,PRS30006013,1998,704.125
105,PRS30006021,2010,17.600
135,PRS30006022,2010,12.500
169,PRS30006023,2014,503.171
207,PRS30006031,2022,20.400
236,PRS30006032,2021,17.100
243,PRS30006033,1998,700.712
297,PRS30006061,2022,38.900


In [5]:
series_id = 'PRS30006032'
period = 'Q01'
best_year = best_value_df[best_value_df['series_id'] == series_id]['year'].values[0]
print(best_year)

2021


In [6]:
merged_df = pd.merge(bls_df, population_df, left_on='year', right_on='ID Year', how='left')[['series_id', 'year', 'period', 'value', 'Population']]

In [7]:
print(merged_df.head())

     series_id  year period  value  Population
0  PRS30006011  1995    Q01    2.6         NaN
1  PRS30006011  1995    Q02    2.1         NaN
2  PRS30006011  1995    Q03    0.9         NaN
3  PRS30006011  1995    Q04    0.1         NaN
4  PRS30006011  1995    Q05    1.4         NaN


In [8]:
merged_df[(merged_df['year']==best_year) & (merged_df['period']==period) & (merged_df['series_id']==series_id)].style.format(precision=1)

,series_id,year,period,value,Population
1152,PRS30006032,2021,Q01,1.2,329725481.0
